In [1]:
# dependencies
import os
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [2]:
#Setup database
app = Flask(__name__)
# app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///ffdb"
# db = SQLAlchemy(app)
ffdb = sqlite3.connect('ffdb')

# Pull data from database into a local variable
# This would be really unacceptable if our database wasn't basically a tiny little spreadsheet
# If complexity increased, it would be very reasonable to tailor select statements to specific requests inside of the functions I call below
cursor = ffdb.cursor()
cursor.execute('''
    SELECT * from fire
''')
data = cursor.fetchall()

In [3]:
# The date fixer function that, frankly, I can't even believe I needed to make
def dateFixer(month, year):
    portugueseMonths = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho',
       'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
    n = 1
    monthDictionary = {}
    
    for monthName in portugueseMonths:
        if len(str(n)) == 1:
            monthDictionary[monthName] = "0" + str(n)
        else:
            monthDictionary[monthName] = str(n)
        n+=1
        
    date = str(year) + "-" + monthDictionary[month] + "-01"
    return date

In [4]:
# a dataframe for your enjoyment
df_data = []
for row in data:
    row = list(row)
    row[4] = dateFixer(row[2],row[0])
    df_data.append(row)

ffdf = pd.DataFrame(df_data, columns=["year","state","month","number","date"])

In [5]:
# Dataframe sorting
ffdf = ffdf.sort_values(by = "date")
data_sorted = []
for r in ffdf.values:
    data_sorted.append(tuple(r))

In [6]:
# Dataframe grouping
state_fire_df = pd.DataFrame()
x = 0
state_list = ffdf['state'].tolist()
ind_state_list = ffdf['state']
ind_state_list = ind_state_list.drop_duplicates()
ind_state_list = ind_state_list.tolist()
for state in ind_state_list:
    x=0
    for fire in state_list:
        if fire == state:
            x=x+1
        else:
            pass
    state_fire_df = state_fire_df.append(pd.DataFrame([{"State":state,"Number of Fires":x}]))
fire_list = state_fire_df["Number of Fires"].to_list()
# Use state list from above and firelist for the tuples.

In [ ]:
# TO DO
# Create a flask app route that gives the necessary JSONified information for making the pie chart work
@app.route("/pie/")
def pie():
    i = 0
    pie_dict= {}
    for state in ind_state_list:
        row = {}
        row["state"] = state
        row["number"] = fire_list[i]
        pie_dict[i] = row
        i += 1
    print(pie_dict)
    return jsonify(pie_dict)

@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")


@app.route("/states")
def states():
    """Return a list of sample names."""
    
    return jsonify(['Acre', 'Alagoas', 'Amapa', 'Amazonas', 'Bahia', 'Ceara', 'Distrito Federal', 'Espirito Santo', 'Goias', 'Maranhao', 'Mato Grosso', 'Minas Gerais', 'Pará', 'Paraiba', 'Pernambuco', 'Piau', 'Rio', 'Rondonia', 'Roraima', 'Santa Catarina', 'Sao Paulo', 'Sergipe', 'Tocantins'])


@app.route("/state_data/<state>")
#  def sample_metadata(sample):
def state_data(state):
   

    """Return the Data for a given state."""
    results = []
    for x in data_sorted:
        if x[1] == state:
            results.append(x)
#     print(results)
    


    # Create a dictionary entry for each row of information
    state_data_dict = {}
    n = 0
    
    # turn each row into its own dictionary and add that to the big dictionary
    for result in results:
        state_data = {}
        state_data["year"] = result[0]
        state_data["state"] = result[1]
        state_data["month"] = result[2]
        state_data["number"] = result[3]
        state_data["date"] = dateFixer(state_data["month"], state_data["year"])
        state_data_dict[n] = state_data
        n += 1
    
#return that info jsonified
#     print(state_data_dict)
    return jsonify(state_data_dict)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Dec/2019 03:28:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2019 03:28:09] "GET /static/js/bonus.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Dec/2019 03:28:09] "GET /static/js/app.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2019 03:28:10] "GET /states HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2019 03:28:10] "GET /state_data/Acre HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2019 03:28:10] "GET /pie/ HTTP/1.1" 200 -


{0: {'state': 'Acre', 'number': 239}, 1: {'state': 'Paraiba', 'number': 478}, 2: {'state': 'Pará', 'number': 239}, 3: {'state': 'Rio', 'number': 717}, 4: {'state': 'Tocantins', 'number': 239}, 5: {'state': 'Minas Gerais', 'number': 239}, 6: {'state': 'Sergipe', 'number': 239}, 7: {'state': 'Mato Grosso', 'number': 478}, 8: {'state': 'Piau', 'number': 239}, 9: {'state': 'Goias', 'number': 239}, 10: {'state': 'Espirito Santo', 'number': 239}, 11: {'state': 'Alagoas', 'number': 240}, 12: {'state': 'Distrito Federal', 'number': 239}, 13: {'state': 'Roraima', 'number': 239}, 14: {'state': 'Ceara', 'number': 239}, 15: {'state': 'Bahia', 'number': 239}, 16: {'state': 'Santa Catarina', 'number': 239}, 17: {'state': 'Amazonas', 'number': 239}, 18: {'state': 'Amapa', 'number': 239}, 19: {'state': 'Sao Paulo', 'number': 239}, 20: {'state': 'Rondonia', 'number': 239}, 21: {'state': 'Maranhao', 'number': 239}, 22: {'state': 'Pernambuco', 'number': 239}}
